# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "chiahong95"
os.environ['KAGGLE_KEY'] = "9d00ef18884366e3c7ff9f5811029d66"
! kaggle competitions download -c ml2021spring-hw2

 96% 250M/261M [00:02<00:00, 143MB/s]
100% 261M/261M [00:02<00:00, 132MB/s]
  0% 0.00/462k [00:00<?, ?B/s]
100% 462k/462k [00:00<00:00, 154MB/s]
 94% 90.0M/95.9M [00:00<00:00, 100MB/s] 
100% 95.9M/95.9M [00:01<00:00, 98.8MB/s]
  0% 0.00/970k [00:00<?, ?B/s]
100% 970k/970k [00:00<00:00, 130MB/s]


In [ ]:
!unzip train_11.npy.zip
!unzip test_11.npy.zip
!unzip train_label_11.npy 

Archive:  train_11.npy.zip
  inflating: train_11.npy            
Archive:  test_11.npy.zip
  inflating: test_11.npy             
Archive:  train_label_11.npy.zip
  inflating: train_label_11.npy      


## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [ ]:
import numpy as np

print('Loading data ...')

train = np.load('train_11.npy')
train_label = np.load('train_label_11.npy')
test = np.load('test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


## Create Dataset

In [ ]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [ ]:
VAL_RATIO = 0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (983945, 429)
Size of validation set: (245987, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [ ]:
BATCH_SIZE = 270

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [ ]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

147

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [ ]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.net = nn.Sequential(                      
            nn.Linear(429, 4096), #layer1
            nn.BatchNorm1d(4096),
            nn.PReLU(), 
            nn.Dropout(0.5),  

            nn.Linear(4096, 2048) ,     
            nn.BatchNorm1d(2048),
            nn.PReLU(), 
            nn.Dropout(0.5),

            nn.Linear(2048, 2048) ,     
            nn.BatchNorm1d(2048),
            nn.PReLU(), 
            nn.Dropout(0.5),

            nn.Linear(2048, 1024) ,     
            nn.BatchNorm1d(1024),
            nn.PReLU(), 
            nn.Dropout(0.5),

            nn.Linear(1024, 1024) ,     
            nn.BatchNorm1d(1024),
            nn.PReLU(), 
            nn.Dropout(0.5),   

            nn.Linear(1024, 512) ,
            nn.BatchNorm1d(512),
            nn.PReLU(),
            nn.Dropout(0.5),

            nn.Linear(512, 256) ,
            nn.BatchNorm1d(256),
            nn.PReLU(),
            nn.Dropout(0.5),

            nn.Linear(256, 128),  #layer8
            nn.BatchNorm1d(128),
            nn.PReLU(),
            nn.Dropout(0.5),

            nn.Linear(128, 39)    #layer output
        )


    def forward(self, x):
        return self.net(x).squeeze(1)

## Training

In [ ]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [ ]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [ ]:
# fix random seed for reproducibility
same_seeds(0)

In [ ]:
# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 100               # number of training epoch
learning_rate = 0.0001      # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

DEVICE: cuda


In [ ]:
# start training
# model.load_state_dict(torch.load('model.ckpt'))
best_acc = 0.0

for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
from google.colab import files
files.download("model.ckpt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [ ]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('pred.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))

In [ ]:
from google.colab import files
files.download("pred.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Post processing**

In [ ]:
import pandas as pd
pred = pd.read_csv('pred.csv')

檢查每個label是否與前或後相同，如果與前後都不相同則以距離比較相近的class取代。

In [ ]:
pre=pred.values[0,1]
for i in range (1,pred.shape[0]-1):
  this=pred.values[i,1]
  next=pred.values[i+1,1]
  if (this!=pre) and (this!=next):
    if abs(this-pre)<abs(this-next):
      pred.values[i,1]=pre
    else:
      pred.values[i,1]=next
  pre=pred.values[i,1]
  i+=1

In [ ]:
pred.to_csv("pred_post.csv",index=False)
files.download("pred_post.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>